<a id="top"></a>
# Pan-STARRS "20 queries" using MAST's TAP service: Filtering


******

## Overview

This notebook is part of a series demonstrating how to address a set of scientific questions using SQL-like Astronomical Data Query Language (ADQL) queries via a Virtual Observatory standard Table Access Protocol (TAP) service at MAST. 

This series aims to be an introduction to how complex queries can be executed using TAP (which might otherwise not be possible to specify with `astroquery`), and to be a resource for how to access MAST databases after the MAST CASJobs service is retired.

These queries are drawn from the 20 queries for SDSS as presented by [Gray, Szalay, et al. (2002)](https://arxiv.org/abs/cs/0202014), adapted for the Pan-STARRS PS1 database.

This notebook presents the subset of queries which are possible through **filtering on column(s) using pre-specified criteria**.


## Learning Goals
By the end of this tutorial, you will:

- Understand how to design and perform filtering queries (with joins) using TAP services, by leveraging multiple ADQL "WHERE" contraints.


****
### Table of Contents

1. [Introduction](#Introduction)
1. [Imports](#Imports)
1. [Connect to TAP service](#Connect-to-TAP-service)
1. [Q2: Find all galaxies with blue surface brightness between 23 and 25 magnitude per square second, and super galactic latitude (sgb) between (-10$^{\circ}$, 10$^{\circ}$), and declination less than zero.](#Q2)
1. [Q4: Find galaxies with an isophotal surface brightness (SB) larger than 24 in the red band, with an ellipticity>0.5, and with the major axis of the ellipse between 30 and 60 arcsec (a large galaxy).](#Q4)
1. [Q5: Find all galaxies with a deVaucouleours profile (r$^{1/4}$ falloff of intensity on disk) and the photometric colors consistent with an elliptical galaxy.](#Q5)
1. [Additional Resources](#Additional-Resources)
1. [About This Notebook](#About-this-Notebook)



*******
## Introduction

Welcome! This notebook demonstrates how to answer example scientific questions by performing SQL-like Astronomical Data Query Language (ADQL) queries accessing the Pan-STARRS catalogs at MAST through a Virual Observatory Table Access Protocol (TAP) service.

As an example, this tutorial will demonstrate how to address 3 scientific questions by combining filtering constraints using ADQL "WHERE" clauses. 

These queries are drawn from (or closely modeled on) the "20 queries for SDSS" presented by [Gray, Szalay, et al. (2002)](https://arxiv.org/abs/cs/0202014). Taken as a whole, this collection provides worked examples on how to leverage relational database capabilities to answer scientific questions, with the aim of providing concrete starting points and references for designing queries for other research applications, both for Pan-STARRS and other large data volume missions (including Roman).

<div class="alert alert-block alert-info">
<b>Note:</b> ADQL/SQL comments follow "--". Comments are used throughout the queries to explain the purpose of specific clauses.
</div>

The workflow for this notebook consists of:
* [Imports](#Imports)
* [Connect to TAP service](#Connect-to-TAP-service)
* [Q2: Find all galaxies with blue surface brightness between 23 and 25 magnitude per square second, and super galactic latitude (sgb) between (-10$^{\circ}$, 10$^{\circ}$), and declination less than zero.](#Q2)
  * [Constructing the query](#Q2:-Constructing-the-query)
  * [Inspecting & visualizing the results](#Q2:-Inspecting-&-visualizing-the-results)
* [Q4: Find galaxies with an isophotal surface brightness (SB) larger than 24 in the red band, with an ellipticity>0.5, and with the major axis of the ellipse between 30 and 60 arcsec (a large galaxy).](#Q4)
  * [Constructing the query](#Q4:-Constructing-the-query)
  * [Inspecting & visualizing the results](#Q4:-Inspecting-&-visualizing-the-results)
* [Q5: Find all galaxies with a deVaucouleours profile (r$^{1/4}$ falloff of intensity on disk) and the photometric colors consistent with an elliptical galaxy.](#Q5)
  * [Constructing the query](#Q5:-Constructing-the-query)
  * [Inspecting & visualizing the results](#Q5:-Inspecting-&-visualizing-the-results)
* [Additional Resources](#Additional-Resources)
* [About This Notebook](#About-this-Notebook)

## Imports
This tutorial makes use of the following libraries: 
- [*numpy*](https://numpy.org/) for numerical calculations
- [*pyvo*](https://pyvo.readthedocs.io) for querying the MAST catalogs via TAP
- [*matplotlib.pyplot*](https://matplotlib.org/stable/api/pyplot_summary.html#module-matplotlib.pyplot) for plotting data
- *time*, *datetime* to determine query duration
- *requests*, *warnings*, *BytesIO*, *PIL*, [*astropy.table Table*](https://docs.astropy.org/en/stable/table/index.html) to fetch & display cutout images of selected objects

In [ ]:
import numpy as np
import pyvo as vo
import matplotlib.pyplot as plt
import datetime
import time

import requests
import warnings
from io import BytesIO
from PIL import Image
from astropy.table import Table

--------
## Connect to TAP service

For all queries, we will be connecting to the Pan-STARRS (PS1) Data release 2 (DR2) catalog. Specifically, we will connect to the new [PS1 DR2 postgres-backed TAP service](https://mast.stsci.edu/vo-tap/api/v0.1/ps1_dr2/), which offers improved performance relative to the legacy database (by factors of 100 or greater, in many cases).

See the [PS1 documentation](link_to_migration_guide_here) for information about the tables available with this new TAP service.
    
<div class="alert alert-warning" style="color:red; background-color:#ffc5c5; border-color:red;">
<b>FIX LINK TO MIGRATION GUIDE ABOVE</b>
</div>

In [ ]:
TAP_service = vo.dal.TAPService(
    "https://mast.stsci.edu/vo-tap/api/v0.1/ps1_dr2"
)

This service supports the following ADQL features, 
and will return up to 100,000 rows.

In [ ]:
TAP_service.describe()

<div class="alert alert-warning" style="color:red; background-color:#ffc5c5; border-color:red;">
The schemas for the PS1 DR2 tables are available at: https://mastdev.stsci.edu/schema_browser/#/
    <p></p>
</div>

<div class="alert alert-warning" style="color:red; background-color:#ffc5c5; border-color:red;">
<i><b>**IF POSSIBLE:**</b></i>
<i>Integrete the schema browser.</i>

<i>Or else provide a simple URL link, so users can still use the schema browser even if it isn't fully integrated with the jupyter notebook?</i>
</div>

*********

## Q2

Following [Gray et al. (2002)](https://arxiv.org/abs/cs/0202014), Query 2 poses: 


> **Find all galaxies with blue surface brightness between 23 and 25 magnitude per square second, and super galactic latitude (sgb) between (-10$^{\circ}$, 10$^{\circ}$), and declination less than zero.**


### Q2: Constructing the query

For this query, we will determine surface brightnesses in the `g` band (the bluest Pan-STARRS filter) using the Kron magnitude from the [`forced_mean_object` table](link_to_migration_guide).  Object positions (necessary for the RA coordinate restriction) are also available in this table (with the updated PS1 DR2 TAP service). 

    
<div class="alert alert-warning" style="color:red; background-color:#ffc5c5; border-color:red;">
Link to migration guide in the above sentence
</div>

It is necessary to join to the `stack_object` table to obtain the Kron radii.

For simplicity, we will also use `raMean` as a proxy instead of using the super-galactic coordinate. (Note that Gray et al. do the same.)

The first constraint

```((fmo.gFKronMag > 0) AND (fmo.gFPSFMag - fmo.gFKronMag > 0.05))```
     
selects for galaxies (excluding point sources).

The following constraints enforce

- the RA range
- the Dec limit
- the blue surface brightness range

We additionally add a constraint on the `stack_object.primaryDetection` key to remove duplicate entries, as `stack_object` contains duplicates of the same object measured in overlapping regions in the stack images.

Finally, to ensure we limit the query to less than the maximum number of entries for a single TAP query, we further restrict the spatial coverage of this query to -30$^{\circ}$$\lesssim$decMean$\lesssim$-28$^{\circ}$.  Here we specify this by restricting the Pan-STARRS `objID` to be less than 74400000000000000 (as the first 5 digits of [Pan-STARRS object identifiers](https://outerspace.stsci.edu/spaces/Pan-STARRS/pages/298812384/PS1+Object+Identifiers) are determined from `floor((decl+90)/0.00833333)`.

In [ ]:
declstart = -30
declend = -28
objidstart = int(np.floor((declstart+90)/0.00833333))
objidend = int(np.floor((declend+90)/0.00833333))
print(objidstart, objidend)

In [ ]:
adql_query = f"""
SELECT fmo.objID, fmo.gFKronMag,
   fmo.gFKronMag + 2.5*log10(PI()*POWER(so.gKronRad,2)) AS bsurfmag, 
   fmo.raMean, fmo.decMean
FROM forced_mean_object as fmo
JOIN stack_object AS so ON 
    fmo.objID=so.objID
WHERE ((fmo.gFKronMag > 0) AND
       (fmo.gFPSFMag - fmo.gFKronMag > 0.05)) -- galaxies
AND fmo.raMean BETWEEN 170 AND 190            -- ra substitute for super-gal coords
AND fmo.decMean < 0
AND fmo.gFKronMag + 2.5*log10(PI()*POWER(so.gKronRad,2))
       BETWEEN 23 AND 25                      -- mag per sq arcsec
AND so.primaryDetection = 1                   -- primary detection in stack_object
AND fmo.objID >= {objidstart}0000000000000            -- select decl >=-30, <-28
AND fmo.objID < {objidend}0000000000000       
"""
print(adql_query)

In [ ]:
start = time.time()
job = TAP_service.run_async(adql_query)
end = time.time()
print(f"Elapsed time: {str(datetime.timedelta(seconds=end-start))}")

### Q2: Inspecting & visualizing the results

This query takes about 1 minute and returned 68,998 rows. 

For the remaining declination range chunked by objID (15 in total), this would thus take ~15 minutes (over all chunks) and return ~1.03 million rows.

In [ ]:
TAP_results = job.to_table()
TAP_results

In [ ]:
# Visualize selected galaxies with a scatter plot, by blue surface brightness:

f, ax = plt.subplots()
pts = ax.scatter(
    TAP_results['raMean'], TAP_results['decMean'],
    s=0.5, lw=0,
    c=TAP_results['bsurfmag'],
    cmap='viridis',
)
ax.set_xlim(ax.get_xlim()[::-1]) # Invert RA axis
ax.set_xlabel("RA")
ax.set_ylabel("Dec")
cbar = f.colorbar(pts)
cbar.set_label("Blue SB")

*********

## Q4

Query 4 poses: 

> **Find galaxies with an isophotal surface brightness (SB) larger than 24 in the red band, with an ellipticity>0.5, and with the major axis of the ellipse between 30 and 60 arcsec (a large galaxy).**


### Q4: Constructing the query

Here we will determine the surface brightness in the `r` band, using Kron magnitudes and radii. 
We will determine ellipticities and major axis sizes from Sersic fits.

This requires combining the following [tables](link_to_migration_guide):

<div class="alert alert-warning" style="color:red; background-color:#ffc5c5; border-color:red;">
Link to migration guide in the above sentence
</div>

- `stack_object`: Kron radius `rKronRad` to get size for surface brightness calculation & size cuts, and `rKronMag` for surface brightness calculation
- `stack_model_fit_ser`: Use the minor/major axis ratio `rSerAb` to get the ellipticity (with $\mathrm{ellipiticy} = \sqrt{1-(b/a)^2}> 0.5$ translating to `rSerAb`=$b/a < \sqrt{0.75}$).

Additionally, we perform quality control by requiring `stack_object.nDetections>3` and `stack_object.nr>1` (number of detections total and in the `r` band), and `stack_object.rpsfQfPerfect>0.9` to weed out some bad objects. We also require `stack_model_fit_ser.rSerChisq>0` to ensure the Sersic fit has at least minimal quality.


Finally, to ensure the query does not exceed the time or maximum row limit for a single TAP query, we further restrict this query to Pan-STARRS slice 16 (by requiring `objID` to be between 129500000000000000 and 133500000000000000).

In [ ]:
adql_query = """
SELECT so.objID, so.raMean, so.decMean, so.rKronRad, so.rKronMag,
   so.rKronMag + 2.5*log10(PI()*POWER(so.rKronRad,2)) as rsurfmag,
   smf.rSerAb, smf.rSerChisq,
   sqrt(1-power(smf.rSerAb,2)) as ellipticity
FROM stack_model_fit_ser AS smf
JOIN stack_object AS so ON 
    smf.objID=so.objID 
    AND smf.uniquePspsSTid=so.uniquePspsSTid
WHERE smf.objID BETWEEN 129500000000000000 AND 133500000000000000 -- select just slice 16
  AND so.objID BETWEEN 129500000000000000 AND 133500000000000000
  AND so.rKronRad BETWEEN 30 AND 60
  AND so.rKronMag > 0
  AND so.rKronMag + 2.5*log10(PI()*POWER(so.rKronRad,2)) < 24 -- mag per sq arcsec
  AND smf.rSerChisq > 0
  AND smf.rSerAb < sqrt(0.75)
  AND so.rpsfQfPerfect > 0.9
  AND so.nDetections > 3
  AND so.nr > 1
"""

In [ ]:
start = time.time()
job = TAP_service.run_async(adql_query)
end = time.time()
print(f"Elapsed time: {str(datetime.timedelta(seconds=end-start))}")

### Q4: Inspecting & visualizing the results

This query takes about 45 seconds to run and returned 67 rows. 

For all 32 slices, this would take a total time of ~24 minutes and return ~2100 rows.

In [ ]:
TAP_results = job.to_table()
TAP_results

To visualize the results, we retreive and cutouts for a subset of these galaxies (using the functions defined below).

In [ ]:
# Adapted from https://spacetelescope.github.io/mast_notebooks/notebooks/Pan-STARRS/PS1_image/PS1_image.html
def get_image_table(ra, dec, filters="grizy"):
    """
    Query ps1filenames.py service to get a list of images
    
    ra, dec = position in degrees
    filters = string with filters to include. includes all by default
    Returns a table with the results
    """
    service = "https://ps1images.stsci.edu/cgi-bin/ps1filenames.py"
    # The final URL appends our query to the PS1 image service
    url = f"{service}?ra={ra}&dec={dec}&filters={filters}"
    # Read the ASCII table returned by the url
    table = Table.read(url, format='ascii')
    return table


def get_imurl(ra, dec, size=240, output_size=None, filters="grizy", color=False):
    """
    Get URL for images in the table
    
    ra, dec = position in degrees
    size = extracted image size in pixels (0.25 arcsec/pixel)
    output_size = output (display) image size in pixels (default = size).
                  output_size has no effect for fits format images.
    filters = string with filters to include. choose from "grizy"
    color = if True, creates a color image (only for jpg or png format).
            Default is return a list of URLs for single-filter grayscale images.   
    Returns a string with the URL
    """
    im_format = "jpg"
        
    # Call the original helper function to get the table
    table = get_image_table(ra, dec, filters=filters)
    url = (f"https://ps1images.stsci.edu/cgi-bin/fitscut.cgi?"
           f"ra={ra}&dec={dec}&size={size}&format={im_format}")
    
    # Append an output size, if requested
    if output_size:
        url = url + f"&output_size={output_size}"
        
    # Sort filters from red to blue
    flist = ["yzirg".find(x) for x in table['filter']]
    table = table[np.argsort(flist)]
    
    if color:
        # We need at least 3 filters to create a color image
        if len(table) < 3:
            raise ValueError("at least three filters are required for an RGB color image")
        # If more than 3 filters, pick 3 filters from the availble results
        if len(table) > 3:
            table = table[[0, len(table)//2, len(table)-1]]
        # Create the red, green, and blue files for our image
        for i, param in enumerate(["red", "green", "blue"]):
            url = url + f"&{param}={table['filename'][i]}"
   
    else:
        # If not a color image, only one filter should be given.
        if len(table) > 1:
            warnings.warn('Too many filters for monochrome image. Using only 1st filter.')
        # Use red for monochrome images
        urlbase = url + "&red="
        url = []
        filename = table[0]['filename']
        url = urlbase+filename
    return url


def get_im(ra, dec, size=240, output_size=None, filters="g", color=False):
    """
    Get image at a sky position. Depends on get_imurl
    
    ra, dec = position in degrees
    size = extracted image size in pixels (0.25 arcsec/pixel)
    output_size = output (display) image size in pixels (default = size).
                  output_size has no effect for fits format images.
    filters = string with filters to include
    Returns the image
    """
    # Image URL
    url = get_imurl(
        ra, dec, size=size, filters=filters, 
        output_size=output_size, color=color
    )

    # JPEG: Request the file, read the bytes
    r = requests.get(url)
    im = Image.open(BytesIO(r.content))
    return im

In [ ]:
# Set image size in pixels (0.25 arcsec/pix)
size = 480

# Get inds for a given pre-selected set of objects
inds = []
ras = [4.57722, 21.64157, 28.70062, 29.83149,
       37.35923, 46.70883, 56.91690, 117.46351,
       129.42904, 132.34119, 136.42660, 138.25536]
decs = [19.39308, 19.83059, 20.79907, 19.00765,
        20.21708, 18.68872, 18.80113, 18.82904,
        20.50383, 19.07499, 18.33797, 20.36527]

for ra, dec in zip(ras, decs):
    inds.append(
        int(np.where(
            (np.abs(TAP_results['raMean']-ra) < 0.00002)
            & (np.abs(TAP_results['decMean']-dec) < 0.00002)
        )[0][0])
    )

f, axes = plt.subplots(3, 4)
f.set_size_inches(16, 12)
axes = axes.flatten()
for i, ind in enumerate(inds):
    ra = TAP_results["raMean"][ind]
    dec = TAP_results["decMean"][ind]
    # Color image
    cim = get_im(ra, dec, size=size, filters="gri", color=True)
    
    # Color image subplot
    sgn = "+"
    if np.sign(dec) < 0:
        sgn = "-"
    axes[i].set_title(f'{ra:0.5f} {sgn}{dec:0.5f} (gri)')
    axes[i].imshow(cim, origin="upper")

    # 7 ticks:
    halfw = int(np.floor(size/2 * 0.25))
    ticklabels = np.linspace(-halfw, halfw, num=7, dtype=int)
    ticklocs = ticklabels / 0.25 + size/2
    axes[i].set_xticks(ticklocs, labels=ticklabels)
    axes[i].set_yticks(ticklocs[::-1], labels=ticklabels)

-------------
## Q5


Query 5 poses: 

> **Find all galaxies with a deVaucouleours profile (r$^{1/4}$ falloff of intensity on disk) and the photometric colors consistent with an elliptical galaxy.**


### Q5: Constructing the query

This query, as implemented in Gray et al. for SDSS, is complex and difficult to replicate exactly (e.g., as the absorption/reddening tables are not available for Pan-STARRS). Thus here we implement similar color cuts to make this example comparable to the SDSS query.

Furthermore, PS1 does not have likelihoods for the de Vaucouleurs versus Sersic morphological fits, so instead heuristic criteria are applied. If the Sersic fit failed, or if it has a higher chi-square value than the de Vaucouleurs fit, then we assume the galaxy is best fit by the de Vaucouleurs profile.

This requires combining the following [tables](link_to_migration_guide):

<div class="alert alert-warning" style="color:red; background-color:#ffc5c5; border-color:red;">
Link to migration guide in the above sentence
</div>

- `stack_object`: Kron magnitudes in `gri` for color & magnitude constraints (used in place of Petrosian magnitudes, as the PS1 documentation states Petrosian magnitudes are unreliable), Kron radius `rKronRad` to get size, Galactic latitude `b`, to avoid the Milky Way Galactic plane (where dust and foreground stars would impact measurements)
- `stack_model_fit_ser`: Sersic fits, including chi-square
- `stack_model_fit_de_v`: de Vaucouleurs fits, including chi-square


Additionally, we perform quality control by requiring `stack_object.nDetections>3` and `stack_object.nr>1` (number of detections total and in the `r` band; and the same in the `g` and `i` bands), and `stack_object.rpsfQfPerfect>0.9` (and the same in the `g` and `i` bands) to weed out some bad objects. We also require `stack_model_fit_de_v.gDevChisq>0` (and similarly for `r` and `i`) to ensure the de Vaucouleurs fit has at least minimal quality. Finally, we use PSF and Kron magnitude differences to select for galaxies (excluding point sources).

Finally, to ensure the query does not exceed the time or maximum row limit for a single TAP query, we further restrict this query to Pan-STARRS slice 16 (by requiring `objID` to be between 129500000000000000 and 133500000000000000).

In [ ]:
adql_query = """
select so.objID, so.raMean, so.decMean, so.b,
   so.rKronRad, so.gKronMag, so.rKronMag, so.iKronMag,
   -- axis ratios and chi-square for de Vaucoulers fits
   fdev.gDevAb, fdev.rDevAb, fdev.iDevAb,
   fdev.gDevChisq, fdev.rDevChisq, fdev.iDevChisq,
   -- axis ratios and chi-square for the Sersic fits
   fser.gSerAb, fser.rSerAb, fser.iSerAb,
   fser.gSerChisq, fser.rSerChisq, fser.iSerChisq
from stack_model_fit_ser as fser
join stack_model_fit_de_v as fdev on fser.objID=fdev.objID and fser.uniquePspsSTid=fdev.uniquePspsSTid
join stack_object as so on fser.objID=so.objID and fser.uniquePspsSTid=so.uniquePspsSTid
where fser.objID between 129500000000000000 and 133500000000000000 -- select just slice 16
  and fdev.objID between 129500000000000000 and 133500000000000000
  and so.objID   between 129500000000000000 and 133500000000000000
  and abs(so.b) > 20                               -- avoid Milky Way plane
  and (so.gKronMag+so.rKronMag+so.iKronMag) > 0 -- require gri detections
  and (so.iPSFMag - so.iKronMag > 0.05)          -- extended sources (galaxies)
  and (so.rKronRad > 0)
  and (fdev.gDevChisq > 0)                         -- de Vaucoleurs fits in gri
  and (fdev.rDevChisq > 0)
  and (fdev.iDevChisq > 0)
  -- require de Vaucoleurs model to be the best fit
  and ((fser.gSerChisq < 0) or (fdev.gDevChisq < fser.gSerChisq))
  and ((fser.rSerChisq < 0) or (fdev.rDevChisq < fser.rSerChisq))
  and ((fser.iSerChisq < 0) or (fdev.iDevChisq < fser.iSerChisq))
  -- quality tests
  and so.gpsfQfPerfect > 0.9
  and so.rpsfQfPerfect > 0.9
  and so.ipsfQfPerfect > 0.9
  and so.nDetections > 3
  and so.nr > 0
  and so.ng > 0
  and so.ni > 0
  -- replace petro mags with Kron mags (PS1 docs say Petro mags are not reliable)
  -- no reddening info in PS1, just set it to zero
  -- use Kron mags for the gri mags too
  -- Use Kron radius instead of petroR50_r (probably not close, but too bad)
  and (so.iKronMag > 17.5)
  and (so.rKronMag > 15.5 OR so.rKronRad > 2)
  and (so.rKronMag < 30 and so.gKronMag < 30 and so.rKronMag < 30 and so.iKronMag < 30)
  and (so.rKronMag < 19.2)
  -- color constraint
  and ( ( (so.rKronMag < (13.1 + (7/3)*(so.gKronMag - so.rKronMag)
                        + 4 *(so.rKronMag - so.iKronMag) -4 * 0.18) )
          and ((so.rKronMag - so.iKronMag - (so.gKronMag - so.rKronMag)/4 - 0.18) BETWEEN -0.2 AND 0.2)
        )
      or
        ( (so.rKronMag < 19.5)
          and ((so.rKronMag - so.iKronMag - (so.gKronMag - so.rKronMag)/4 - 0.18)
               > (0.45 - 4*(so.gKronMag - so.rKronMag)))
          and ((so.gKronMag - so.rKronMag) > (1.35 + 0.25 *(so.rKronMag - so.iKronMag)))
        )
    )
"""

In [ ]:
start = time.time()
job = TAP_service.run_async(adql_query)
end = time.time()
print(f"Elapsed time: {str(datetime.timedelta(seconds=end-start))}")

### Q5: Inspecting & visualizing the results

This query takes about 45 seconds to run and returned 527 rows. 

For all 32 slices, this would take a total time of ~24 minutes and return ~16,900 rows.

In [ ]:
TAP_results = job.to_table()
TAP_results

To visualize the results, again we retreive and cutouts for a subset of these galaxies.

In [ ]:
inds = []

# Get the same curated list from RW
imags = [17.50, 17.78, 17.93, 18.05,
         18.15, 18.21, 18.29, 18.36, 
         18.41, 18.49, 18.55, 18.68]
ras = [20.14718, 357.07376, 49.39537, 214.34062,
       37.13989, 223.76736, 133.60074, 225.50517,
       60.74114, 116.48517, 207.24531, 9.84269]
for ra, imag in zip(ras, imags):
    inds.append(
        int(np.where(
            (np.abs(TAP_results['iKronMag']-imag) < 0.02)
            & (np.abs(TAP_results['raMean']-ra) < 0.00002)
        )[0][0])
    )
    
# Set image size 
size = 126 # a bit more than 0.5 arcmin

f, axes = plt.subplots(3, 4)
f.set_size_inches(16, 12)
axes = axes.flatten()
for i, ind in enumerate(inds):
    ra = TAP_results["raMean"][ind]
    dec = TAP_results["decMean"][ind]
    # Color image
    cim = get_im(ra, dec, size=size, filters="gri", color=True)
    
    # Color image subplot
    sgn = "+"
    if np.sign(dec) < 0:
        sgn = "-"
    axes[i].set_title(
        f'{ra:0.5f} {sgn}{dec:0.5f} i={TAP_results["iKronMag"][ind]:0.2f} (gri)'
    )
    axes[i].imshow(cim, origin="upper")

    # 7 ticks:
    halfw = int(np.floor(size/2 * 0.25))
    ticklabels = np.linspace(-halfw, halfw, num=7, dtype=int)
    ticklocs = ticklabels / 0.25 + size/2
    axes[i].set_xticks(ticklocs, labels=ticklabels)
    axes[i].set_yticks(ticklocs[::-1], labels=ticklabels)

----------

## Additional Resources

### Table Access Protocol

- IVOA standard for RESTful web service access to tabular data
- http://www.ivoa.net/documents/TAP/

### Pan-STARRS 1 DR 2

- https://outerspace.stsci.edu/display/Pan-STARRS/

### Astronomical Query Data Language (2.0)

- IVOA standard for querying astronomical data in tabular format, with geometric search support
- http://www.ivoa.net/documents/latest/ADQL.html

### PyVO

- an affiliated package for [astropy](https://www.astropy.org/)
- find and retrieve astronomical data available from archives that support standard IVOA virtual observatory service protocols.
- https://pyvo.readthedocs.io/en/latest/index.html


### Full list of MAST/TAP services
- A full list of available MAST TAP services can be found at:
- https://mast.stsci.edu/vo-tap


## Citations
If you use `astropy` for published research, please cite the
authors. Follow these links for more information about citing `astropy`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)

If you use Pan-STARRS data accessed through MAST for published research, 
please include the following acknowledgements, found at the following links:

* [Acknowledging Pan-STARRS](https://archive.stsci.edu/publishing/mission-acknowledgements#section-895d38a0-86b3-4143-b521-6cc3312701f9)
* [Acknowledging MAST](https://archive.stsci.edu/gsc/mast_data_use.html)


## About this Notebook

**Author(s):**  Rick White, Sedona Price<br>
**Keyword(s):** Tutorial, TAP, pyvo, ADQL, Pan-STARRS <br>
**First Published:** 2026-01-07 <br>
**Last Updated:** 2026-01-07
***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/style-guides/master/guides/images/stsci-logo.png" alt="Space Telescope Logo" width="200px"/> 